# HW 3: Classification, Evaluation, and Deployment

In this homework, you will experience a complete machine learning cycle from data preparation to deployment. You will prepare a dataset, make a model, evaluate models to find the best fit, and deploy it to a simple web page. Our main objective is to make you try classification and evaluation methods, so we will only apply essential data preprocessing techniques but mainly focus on classification and evaluation.

We will use the **Adult** dataset from the UCI repository and more information about the data is available [here](http://archive.ics.uci.edu/ml/datasets/Adult). Since we have removed and changed the dataset for a grading purpose, use the one that we provide on ilearn.

The dataset contains the information to check whether income exceeds $50K/yr based on census data. The datasets consist of 14 attributes and one binary class variable:

- income: >50K, <=50K

- age: continuous.

- workclass: Private, Self-emp-not-inc, Self-emp-inc, Federal-gov, Local-gov, State-gov, Without-pay, Never-worked.

- fnlwgt: continuous.

- education: Bachelors, Some-college, 11th, HS-grad, Prof-school, Assoc-acdm, Assoc-voc, 9th, 7th-8th, 12th, Masters, 1st-4th, 10th, Doctorate, 5th-6th, Preschool. education-num: continuous.

- education-num: continuous.

- marital-status: Married-civ-spouse, Divorced, Never-married, Separated, Widowed, Married-spouse-absent, Married-AF-spouse.

- occupation: Tech-support, Craft-repair, Other-service, Sales, Exec-managerial, Prof-specialty, Handlers-cleaners, Machine-op-inspct, Adm-clerical, Farming-fishing, Transport-moving, Priv-house-serv, Protective-serv, Armed-Forces.

- relationship: Wife, Own-child, Husband, Not-in-family, Other-relative, Unmarried.

- race: White, Asian-Pac-Islander, Amer-Indian-Eskimo, Other, Black.

- sex: Female, Male.

- capital-gain: continuous.

- capital-loss: continuous.

- hours-per-week: continuous.

- native-country: United-States, Cambodia, England, Puerto-Rico, Canada, Germany, Outlying-US(Guam-USVI-etc), India, Japan, Greece, South, China, Cuba, Iran, Honduras, Philippines, Italy, Poland, Jamaica, Vietnam, Mexico, Portugal, Ireland, France, Dominican-Republic, Laos, Ecuador, Taiwan, Haiti, Columbia, Hungary, Guatemala, Nicaragua, Scotland, Thailand, Yugoslavia, El-Salvador, Trinadad&Tobago, Peru, Hong, Holand-Netherlands.

and we have a binary class which can be `>=50` or `<50`.

**NOTE**
- Unlike the labs, each function you make here will be **graded**, so it is important to *strictly* follow the instruction.
- **Import** all necessary libraries yourself whenever needed. Failure to run any code can affect your grade.

## Contents

- **Total points: 6.0 pt**

0. Preparation (0.7 pt)
 - Task 1: Drop missing values (0.1 pt).
 - Task 2: Assign X and y (0.1 pt).
 - Task 3: One-hot encoding (0.1 pt).
 - Task 4: Train test split (0.2 pt).
 - Task 5: Standardization (0.2 pt).
1. Classification (2.9 pt)
 - Task 6: Random forest (0.5 pt).
 - Task 7: SVM with diverse kernels (0.4 pt).
 - Task 8: Decision tree and Random Forest (2.0 pt).
2. Evaluation (1.8 pt)
 - Task 9: Accuracy, Precision, Recall, F1-score (0.3 pt).
 - Task 10: AUC/AUPRC (0.3 pt).
 - Task 11: Apply them together with scikit-learn (0.4 pt).
 - Task 12: Manual implementation of performance metrics (0.8 pt).
3. Deployment (0.6 pt)
 - Task 13: Save models into a file using pickle (0.3 pt).
 - Task 14: DASH deployment (0.3 pt).

# 0. Preparation

##### Student information

Please provide your information for automatic grading.

In [5]:
STUD_SUID = 'maha5381'
STUD_NAME = 'Majid Hassanabadi'
STUD_EMAIL = 'maha5381@dsv.su.se'

#### Basic libraries

These libraries will be frequently used throughout the homework. Do not change the block below.

In [6]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pickle
RANDOM_STATE = 12345 #Do not change it!
np.random.seed(RANDOM_STATE) #Do not change it!

#### Load the dataset

Use the **Adult** dataset located ilearn, and load it here using pandas. 

In [3]:
adult = pd.read_csv("datasets/adult.data", sep=",", header=None, skipinitialspace=True)

You can run the line below to give the dataframe proper column names.

In [4]:
adult.columns = ['age', 'workclass', 'fnlwgt', 'education', 'education-num', 'marital-status', 'occupation', 'relationship', 'race',  'sex', 'capital-gain', 'capital-loss', 'hours-per-week', 'native-country', 'income']

Here you can find out some basic information by calling *info(), head()*, and *describe()*.

In [5]:
adult.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32561 entries, 0 to 32560
Data columns (total 15 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   age             32561 non-null  int64 
 1   workclass       32561 non-null  object
 2   fnlwgt          32561 non-null  int64 
 3   education       32561 non-null  object
 4   education-num   32561 non-null  int64 
 5   marital-status  32561 non-null  object
 6   occupation      32561 non-null  object
 7   relationship    32561 non-null  object
 8   race            32561 non-null  object
 9   sex             32561 non-null  object
 10  capital-gain    32561 non-null  int64 
 11  capital-loss    32561 non-null  int64 
 12  hours-per-week  32561 non-null  int64 
 13  native-country  32561 non-null  object
 14  income          32561 non-null  object
dtypes: int64(6), object(9)
memory usage: 3.7+ MB


In [6]:
adult.head()

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,income
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K


In [7]:
adult.describe()

,age,fnlwgt,education-num,capital-gain,capital-loss,hours-per-week
count,32561.000000,3.256100e+04,32561.000000,32561.000000,32561.000000,32561.000000
mean,38.581647,1.897784e+05,10.080679,1077.648844,87.303830,40.437456
std,13.640433,1.055500e+05,2.572720,7385.292085,402.960219,12.347429
min,17.000000,1.228500e+04,1.000000,0.000000,0.000000,1.000000
25%,28.000000,1.178270e+05,9.000000,0.000000,0.000000,40.000000
50%,37.000000,1.783560e+05,10.000000,0.000000,0.000000,40.000000
75%,48.000000,2.370510e+05,12.000000,0.000000,0.000000,45.000000
max,90.000000,1.484705e+06,16.000000,99999.000000,4356.000000,99.000000


It seems like there is no null data. However, if you read the description of the dataset, it says that there are missing parts represented as "?". You can count them by using the same technique we used for checking nulls in the previous lab. We have missing values in specific columns only, and it is about 5% of data records.

In [8]:
(adult == "?").sum()

age                  0
workclass         1836
fnlwgt               0
education            0
education-num        0
marital-status       0
occupation        1843
relationship         0
race                 0
sex                  0
capital-gain         0
capital-loss         0
hours-per-week       0
native-country     583
income               0
dtype: int64

#### Task 1: Drop missing values (0.1 pt)

There are many ways to handle missing values, such as imputation or putting median/mean values, but we will practice the simplest way: removing the rows with missing values.

- Complete the function below which removes any rows with missing values.

In [9]:
def drop_missing_values(df, miss):
    """
    Input: 
      df: the dataframe (adult in our case)
      miss: a character to represent missing value ("?" in our case)
      
    Output: the dataframe without the missing values

    Step 1: Replace the value 'miss' with np.nan.
    Step 2: Drop the nan values and store the result in data_dropped.
    Step 3: Return data_dropped
    
    """

    # for row in range(df.shape[0]):
    #   for column in range(df.shape[1]):
    #     if df.iloc[row][column] == miss:
    #       df.iloc[row][column] = np.nan
    data_dropped = df.replace(miss, np.nan)
    data_dropped.dropna(inplace = True)
    
     # CHANGE IT
    return data_dropped

Apply your function to our dataset `adult` and save the result to `adult_dropped`.

In [10]:
adult_dropped = drop_missing_values(adult, "?")

The output of the function should have the same attributes but only less number of the rows. Check how many rows are removed. Your dataset should have 30,162 rows!

In [11]:
adult_dropped.shape

(30162, 15)

#### Task 2: Assign X and y (0.1 pt)

Then let's split our dataset into two parts (`X` for attributes and `y` for labels) to use scikit-learn's various methods.
- Use `adult_dropped`.
- `X` should have all the attributes without the labels (the last column).
- `y` should be a Pandas Series only with the labels.

In [12]:
X = adult_dropped.iloc[:,:-1] # CHANGE IT!
y = adult_dropped.iloc[:,-1] # CHANGE IT!

Check the type and size here.

In [13]:
(X.shape, y.shape, type(X), type(y))

((30162, 14), (30162,), pandas.core.frame.DataFrame, pandas.core.series.Series)

#### Task 3: One-hot encoding (0.1 pt)

Unfortunately, scikit-learn does not support categorical attributes very well even for decision tree, and that means we need to convert them into reasonal form of numerical data to fit the algorithms. There is one way called one-hot encoding, which transforms the categorical data into multiple numerical columns for each possible value. There are various ways to apply this, especially using [scikit-learn](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.OneHotEncoder.html) or [pandas](https://pandas.pydata.org/docs/reference/api/pandas.get_dummies.html) but here we will use the pandas function to keep the dataframe structure.

- Finish one_hot_encoding function which applies one-hot encoding to a given dataframe.


In [14]:
def one_hot_encoding(df):
    """
    Input:
        df: the attributes (X in our case)
    Output: one-hot encoded dataframe
    
    Step 1: Use pd.get_dummies to convert df to a one-hot-encoded form. 
            Enable an option called drop_first to remove duplication.
    Step 2: Return the one-hot-encoded dataframe.
    
    * Those steps and suggested method are just for your convenience. You can use your own choice of methods.
    """
    
    df_onehot = pd.get_dummies(df, drop_first = True) # CHANGE IT
    return df_onehot

- Create `X_onehot` by calling `one_hot_encoding` function with `X`.

In [15]:
X_onehot = one_hot_encoding(X)

Check your result by calling any methods you learned. If you successfully followed the instruction, the output (`X_onehot`) should have 96 columns.

In [16]:
X_onehot.head()

,age,fnlwgt,education-num,capital-gain,capital-loss,hours-per-week,workclass_Local-gov,workclass_Private,workclass_Self-emp-inc,workclass_Self-emp-not-inc,...,native-country_Portugal,native-country_Puerto-Rico,native-country_Scotland,native-country_South,native-country_Taiwan,native-country_Thailand,native-country_Trinadad&Tobago,native-country_United-States,native-country_Vietnam,native-country_Yugoslavia
0,39,77516,13,2174,0,40,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
1,50,83311,13,0,0,13,0,0,0,1,...,0,0,0,0,0,0,0,1,0,0
2,38,215646,9,0,0,40,0,1,0,0,...,0,0,0,0,0,0,0,1,0,0
3,53,234721,7,0,0,40,0,1,0,0,...,0,0,0,0,0,0,0,1,0,0
4,28,338409,13,0,0,40,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0


In [17]:
X_onehot.shape

(30162, 96)

#### Task 4: Train test split (0.2 pt)

We also need to split our dataset further into four parts for evaluation.

- Use scikit-learn's `train_test_split` function to divide the dataset into four parts.
- Follow the instruction below carefully to get a point!.
    - Use `X_onehot` and `y`.
    - Assign 30% to a test set.
    - Use our random state (`RANDOM_STATE`)
    - Enable stratify option.

In [18]:
# Remove the assigned values and write train_test_split function
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_onehot, y, test_size=0.3, random_state=RANDOM_STATE, stratify=y)

Check the type and size here.

In [19]:
(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

((21113, 96), (9049, 96), (21113,), (9049,))

#### Task 5: Standardization (0.2 pt)

After removing the missing value and split X and y, we need to take care of our numerical attributes. As you can check from `describe()` function, we have five numerical attributes and they all have different mean and standard deviation. Not all machine learning models require standardization of numerical attributes, but some do. In this homework, SVM might be the case that the standardization is required. It might be better to make standardized version when performing data preparation. 

- One-hot encoded data does not need to be standardized! So you need to choose the numerical columns only
 - ["age", "fnlwgt",  "education-num", "capital-gain", "capital-loss", "hours-per-week"]
 
- For this task, you need to import sklearn's `StandardScaler`.

In [20]:
from sklearn.preprocessing import StandardScaler

In [21]:
def standardize(X_train, X_test, numerical):
    """
    Input:
        - X_train: A split training set from Task 4
        - X_test: A split test set from Task 5
        - numerical: Numerical columns that should be standardized
    Output:
        - X_train_st: A standardized numerical attributes (ndarray)
        - X_test_st: A standardized numerical attributes (ndarray)
    
    Step 1: Initialize StandardScaler.
    Step 2: Create X_train_numerical, X_test_numerical by selecting numerical columns from original X_train X_test.
    Step 3: Fit StandardScaler on X_train. You should only use the numerical columns only.
    Step 4: Use trained StandardScaler and run transform function both on X_train_st (for the training set) 
            and X_test_st (for the test set). This job will standardize both training and test sets based on
            the statistics of training set. You should only use numerical attributes.
    Step 5: Return X_train_st, X_test_st
    
    """
    
    # Step 1
    sc = StandardScaler() # CHANGE IT
    
    # Step 2
    X_train_numeric = X_train[numerical] # CHANGE IT
    X_test_numeric = X_test[numerical] # CHANGE IT
    
    # Step 3
    scaler = sc.fit(X_train_numeric)
    
    # Step 4
    # Assign two outputs of transformation function to X_train_st (for the training set) and X_test_st (for the test set)
    X_train_st = scaler.transform(X_train_numeric) # CHANGE IT
    X_test_st = scaler.transform(X_test_numeric) # CHANGE IT
    
    # Step 5
    # Note that those two variable should only contain numerical attributes, not the whole ones.
    return X_train_st, X_test_st

In [22]:
def standardize_wrapper(X_train, X_test, numerical):
    # DO NOT CHANGE THIS FUNCTION
    # This function is to ensure that the datasets keep the Pandas DataFrame format.
    if X_train.shape == (0, 0): return pd.DataFrame([0]), pd.DataFrame([0])
    
    X_train_st, X_test_st = standardize(X_train, X_test, numerical)
    
    X_train_st_df = X_train.copy()
    X_train_st_df[numerical] = X_train_st
    X_test_st_df = X_test.copy()
    X_test_st_df[numerical] = X_test_st
    
    return X_train_st_df, X_test_st_df

The line below will apply your standardization function to the datasets. Run the block and check the result. 
DO NOT change `standardize_wrapper`, just implement `standardize`.

In [23]:
# Make a list for numerical attributes: "age", "fnlwgt", "education-num", "capital-gain", "capital-loss", "hours-per-week"
numerical = ["age", "fnlwgt",  "education-num", "capital-gain", "capital-loss", "hours-per-week"] # CHANGE IT
X_train_st, X_test_st = standardize_wrapper(X_train, X_test, numerical)

Your numerical attributes (["age", "fnlwgt", "education-num", "capital-gain", "capital-loss", "hours-per-week"]) should have near zero mean and one standard deviation.

In [24]:
X_train_st.describe()

,age,fnlwgt,education-num,capital-gain,capital-loss,hours-per-week,workclass_Local-gov,workclass_Private,workclass_Self-emp-inc,workclass_Self-emp-not-inc,...,native-country_Portugal,native-country_Puerto-Rico,native-country_Scotland,native-country_South,native-country_Taiwan,native-country_Thailand,native-country_Trinadad&Tobago,native-country_United-States,native-country_Vietnam,native-country_Yugoslavia
count,2.111300e+04,2.111300e+04,2.111300e+04,2.111300e+04,2.111300e+04,2.111300e+04,21113.000000,21113.000000,21113.000000,21113.000000,...,21113.000000,21113.000000,21113.000000,21113.000000,21113.000000,21113.000000,21113.000000,21113.000000,21113.000000,21113.000000
mean,2.090562e-16,-2.265222e-17,-7.976747e-16,2.751185e-16,-9.892885e-16,-7.890429e-16,0.069673,0.739734,0.034718,0.081040,...,0.001374,0.003694,0.000332,0.002416,0.001326,0.000568,0.000474,0.911524,0.001658,0.000568
std,1.000024e+00,1.000024e+00,1.000024e+00,1.000024e+00,1.000024e+00,1.000024e+00,0.254601,0.438790,0.183069,0.272903,...,0.037037,0.060671,0.018206,0.049090,0.036394,0.023834,0.021759,0.283993,0.040683,0.023834
min,-1.633523e+00,-1.674023e+00,-3.581744e+00,-1.468402e-01,-2.159858e-01,-3.313280e+00,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,-7.973943e-01,-6.843151e-01,-4.404088e-01,-1.468402e-01,-2.159858e-01,-7.607792e-02,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000
50%,-1.132886e-01,-1.077691e-01,-4.774195e-02,-1.468402e-01,-2.159858e-01,-7.607792e-02,0.000000,1.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000
75%,6.468289e-01,4.460466e-01,1.130259e+00,-1.468402e-01,-2.159858e-01,3.389479e-01,0.000000,1.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000
max,3.915334e+00,1.232133e+01,2.308259e+00,1.334760e+01,9.587298e+00,4.821227e+00,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


After finishing a simple data processing, let's proceed to our main task, classification.

# 1. Classification

In this assignment, we will run random forest (RF), and support vector machine (SVM) with different kernels using scikit-learn. Then we will implement score functions for decision trees and main functions for random forests to understand the concepts better. We will continue to use the pre-processed Adult dataset from the section zero (Task 1-5).

#### Task 6: Random forest (graded, 0.5 pt)

Here you will run the random forest algorithm using scikit-learn, together with cross-validation. Detailed information about the random forest in scikit-learn can be found [here](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html).

Your task is as follows:
 1. Create a random forest classifier `rf` with the random state defined above (`RANDOM_STATE`). Do not specify any other parameters.
 2. Report an average cross-validation score `rf_cross_val_score` with stratified k-fold with **cv=5**. You should report the average score, not a list of the scores. Use `X_onehot` and `y`, not the training or test set (0.2 pt). 
 - ***Note that you are reporting an average cross validation score, not a list of scores.***

 3. Run grid search `gs` with a single dictionary `grid_dict` with two keys 1) max_depth from 1 to 3 (included), and 2) min_samples_split from 2 to 4 (included) and report the best classifier into the variable `rf_best_classifier`. Set **cv=5** for grid search cross-validation. Since grid search uses stratified k-fold inside, you should put a complete dataset, not a split training set (Use `X_onehot` and `y`). This task can take more than five minutes depending on computing power (0.3 pt).

In [3]:
grid_dict ={'max_depth' : [1,2,3] , 'min_samples_split' : [2,3,4]} # CHANGE IT

In [4]:
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier(random_state=RANDOM_STATE) # CHANGE IT

NameError: name 'RANDOM_STATE' is not defined

In [ ]:
from sklearn.model_selection import cross_val_score
rf_cross_val_score =(cross_val_score(rf, X_onehot, y, cv=5)).mean() # CHANGE IT

Run this line to check your score. Your score should be above 0.80.

In [ ]:
rf_cross_val_score

0.8500763484302297

In [ ]:
from sklearn.model_selection import GridSearchCV
gs = GridSearchCV(rf, grid_dict, cv=5)
gs.fit(X_onehot,y)

GridSearchCV(cv=5, estimator=RandomForestClassifier(random_state=12345),
             param_grid={'max_depth': [1, 2, 3],
                         'min_samples_split': [2, 3, 4]})

In [ ]:
rf_best_classifier = gs.best_estimator_ # CHANGE IT

Report your best classifier here.

In [ ]:
rf_best_classifier

RandomForestClassifier(max_depth=3, random_state=12345)

#### Task 7: SVM with diverse kernels (graded, 0.4 pt)

We already tried a simple SVC with the RBF kernel before. Here you will run SVM again, but with different kernels, and together with cross-validation. Detailed information about SVC in scikit-learn can be found [here](https://scikit-learn.org/stable/modules/generated/sklearn.svm.SVC.html#sklearn.svm.SVC).

Your task is as follows:

  1. Create a standard SVC classifier `svc` without setting any parameter.
  2. Report a test score to `svm_ho_score` using your standardized training set `X_train_st`. and test set `X_test_st` (0.1 pt).
  3. Run grid search with a list of two parameter dictionaries, one with kernel = ['linear', 'poly', 'rbf'] and the other one with C = [1, 10, 100]. This means you have to create a list containing two different dictionaries. Report the best classifier into the variable `svm_best_classifier`. Set **cv=3** for grid search cross-validation. Use `X_train_st` and `y_train`. This task can take more than five minutes depending on computing power (0.3 pt).

In [ ]:
from sklearn.svm import SVC
svc = SVC()
svc.fit(X_train_st, y_train) # CHANGE IT

SVC()

In [ ]:
svm_ho_score = svc.score(X_test_st, y_test) # CHANGE IT

Run this line to check your score. Your score should be above 0.80.

In [ ]:
svm_ho_score

0.8540170184550779

In [ ]:
grid_dict = [{'kernel' : ['linear', 'poly', 'rbf']} , {'C' : [1, 10, 100]}] # CHANGE IT
gs = GridSearchCV(svc, grid_dict, cv=3)
gs.fit(X_train_st,y_train) # CHANGE IT
svm_best_classifier = gs.best_estimator_ # CHANGE IT

In [ ]:
svm_best_classifier

SVC()

#### Task 8: Decision tree and Random Forest (2.0 pt)

We will now implement a few modules for decision tree and random forest. Follow the instruction carefully so that you can return a correct result. This task is composed of two subtasks as follows:

  - 8-1. Entropy, gini index, and information gain (0.7 pt)
  - 8-2. Random forest implementation (1.3 pt)

First section of this task is to create three functions used to evaluate and grow the tree, which are covered in the lecture. Entropy, gini index are two main scores used for it. Information gain is the final score to choose a feature for dividing the node. Those scores are essencial for decision tree to work properly and a wrong score can lead to choosing the features that are not proper for creating a high-performing tree.

- For simplicity, you will not use the **adult** dataset in this task but will use a simple **playgolf** dataset with categorical attributes.

- Task 8 is a continuous task and the grade is evaluated by the result of the function. Since one function calls other functions in the task, failing to develop one function can affect the whole grade.

Import playgolf dataset to `playgolf`. You can find it in the homework file.

In [7]:
playgolf = pd.read_csv('datasets/playgolf.csv')

In [8]:
playgolf.columns

Index(['Outlook', 'Temp', 'Humidity', 'Windy', 'Play Golf'], dtype='object')

In [9]:
playgolf.head()

,Outlook,Temp,Humidity,Windy,Play Golf
0,Rainy,Hot,High,False,No
1,Rainy,Hot,High,True,No
2,Overcast,Hot,High,False,Yes
3,Sunny,Mild,High,False,Yes
4,Sunny,Cool,Normal,False,Yes


- Subtask 1: Create a gini index function.
 - The function receives a list and calculate a gini index. The Gini Index is calculated by subtracting the sum of the squared probabilities of each class from one. 
 - You should double check the lecture slides and the examples below to make sure you made a correct function.
 - You can use collections.Counter() to count labels of the dataset.

In [10]:
from collections import Counter

def gini(dataset):
    """
    A function that calculates gini index of a given list.
    
    Input
     - dataset: a list of labels.
    Output
     - impurity: gini index of the list.
    
    You do not need to keep the output name of this function, the grade only depends on the correct outputs.
    
    """

    counter_dict = Counter(dataset)
    sigma_p = 0
    ds_length = len(dataset)
    if ds_length != 0:
        for element in counter_dict.keys():
            sigma_p += (counter_dict[element]/ds_length)**2
    impurity = 1-sigma_p
    return impurity

Your gini index is expected to have the following results:
- `0.5` for `[0,0,1,1]`
- `0.4082` for `[0,0,0,0,0,1,1]`

In [11]:
gini([0,0,1,1])

0.5

In [12]:
gini([0,0,0,0,0,1,1])

0.40816326530612246

Report a gini score of the `Temp` attribute of **playgolf** to `gini_score` (0.2 pt).

In [13]:
gini_score = gini(playgolf['Temp']) # CHANGE IT

Print your score here!

In [14]:
gini_score

0.653061224489796

- Subtask 2: Create an entropy function (0.2 pt).
 - You should double check the lecture slides and the examples below to make sure you made a correct function.
 - You can use collections.Counter() to count labels of the dataset.

In [15]:
def entropy(dataset):
    """
    A function that calculates gini index of a given list.
    
    Input
     - dataset: a list of labels.
    Output
     - impurity: entropy value of the list.
    
    You do not need to keep the output name of this function, the grade only depends on the correct outputs.
    
    """
    import math

    counter_dict = Counter(dataset)
    sigma_p = 0
    ds_length = len(dataset)
    if ds_length != 0:
        for element in counter_dict.keys():
            p=counter_dict[element]/ds_length
            sigma_p += (p)*math.log(p,2)
    impurity = -sigma_p
    return impurity

Your entropy is expected to have the following results:
- `1.0` for `[0,0,1,1]`
- `0.8631` for `[0,0,0,0,0,1,1]`

In [16]:
entropy([0,0,1,1])

1.0

In [17]:
entropy([0,0,0,0,0,1,1])

0.863120568566631

Report a gini score of the `Windy` attribute of **playgolf** to `entropy_score` (0.2 pt).

In [18]:
entropy_score = entropy(playgolf['Windy']) # CHANGE IT

Print your score here!

In [19]:
entropy_score

0.9852281360342516

- Subtask 3: Create an information gain function. 

  - **DO NOT use entropy but only use the gini index for scores.**
  - Check the lecture slides and the examples below to make sure you made a correct function.

In [20]:
def information_gain(labels_start, labels_split):
    """
    Calculate information gain when we have an information of label distribution before and after split operation.
    This information gain function receives two values:
    
    Input:
      - labels_start: A single list of all current labels
        e.g.) [0,0,0,0,1,1,1,1]
      - labels_split: A list of lists representing split 
        e.g.) [ [0,0,1,1], [1,1,0,0] ]
    
    Then we can calculate information gain by calculating the gini index before splitting,
    and substract (gini index * proportion of the subset) for each list after splitting from there.
    
    Output:
      - info_gain: Information gain
    
    You do not need to keep the output name of this function, the grade only depends on the correct outputs.
    
    """
    start_gini = gini(labels_start)
    split_gini = 0
    for item in labels_split:
      split_gini += len(item)/len(labels_start)*gini(item)
    info_gain = start_gini-split_gini # CHANGE IT
    return info_gain

In [21]:
information_gain([0,0,0,0,1,1,1,1], [[0,0,1,0],[1,1,0,1]])

0.125

Your information gain is expected to have the following results:
- `0.0` for `[0,0,0,0,1,1,1,1], [[0,0,1,1],[0,0,1,1]]`
- `0.5` for `[0,0,0,0,1,1,1,1], [[0,0,0,0],[1,1,1,1]]`
- `0.125` for `[0,0,0,0,1,1,1,1], [[0,0,1,0],[1,0,1,1]]`

Here we have labels before and after splitting information. Use those two values to calculate information gain and report it to `info_gain_score` using your own `information_gain` function (0.2 pt).

In [22]:
labels_start = [1,2,1,2,2,1,2,1,3,3,3]
labels_split = [[3,3,3],[1,2,1,1],[2,2,1,2]]

In [23]:
info_gain_score = information_gain(labels_start, labels_split)

Print your score here!

In [24]:
info_gain_score

0.38842975206611574

Now it is time for random forests. We will give you a basic `split` function for the algorithms you will develop. This split function receives the attributes (`X`), the label (`y`) and one `feature` (attribute) of it, and split the whole dataset based on the categories of the selected feature and return split data subsets and label subsets. Using those split values, you are going to make few functions needed for a random forest. 

- Note that this assignment does not make a whole working random forest but the core functions to understand the algorithm.

In [25]:
def split(X, y, attr):
    split_attrs = []
    split_labels = []
    
    for val in X[attr].unique():
        attr_subset = []
        label_subset = []
        
        for idx, row in X.iterrows():
            
            if row[attr] == val:
                attr_subset.append(row)
                label_subset.append(y[idx])
                
        split_attrs.append(pd.DataFrame(attr_subset))
        split_labels.append(label_subset)
        
    return split_attrs, split_labels

Check out the result by running the function below and also check the Windy column to understand what the function does.

In [26]:
split(playgolf.drop('Play Golf', axis=1), playgolf['Play Golf'], 'Windy')

([     Outlook  Temp Humidity  Windy
  0      Rainy   Hot     High  False
  2   Overcast   Hot     High  False
  3      Sunny  Mild     High  False
  4      Sunny  Cool   Normal  False
  7      Rainy  Mild     High  False
  8      Rainy  Cool   Normal  False
  9      Sunny  Mild   Normal  False
  12  Overcast   Hot   Normal  False,
       Outlook  Temp Humidity  Windy
  1      Rainy   Hot     High   True
  5      Sunny  Cool   Normal   True
  6   Overcast  Cool   Normal   True
  10     Rainy  Mild   Normal   True
  11  Overcast  Mild     High   True
  13     Sunny  Mild     High   True],
 [['No', 'Yes', 'Yes', 'Yes', 'No', 'Yes', 'Yes', 'Yes'],
  ['No', 'No', 'Yes', 'Yes', 'Yes', 'No']])

In [27]:
playgolf['Windy']

0     False
1      True
2     False
3     False
4     False
5      True
6      True
7     False
8     False
9     False
10     True
11     True
12    False
13     True
Name: Windy, dtype: bool

Subtask 4: Now we can make a function for choosing the best feature to split, given the dataset of the node (It will be a full dataset if we run this function on the root node.). The function `best_split` receives the datasets (`X`, `y`) and the number of attributes to choose from the dataset, and returns the best feature among the chosen one and it's information gain. This process is one of the core processes of the random forest. 

In [39]:
# 0.4 pt
def best_split(X, y, num_attr):
    """
    Input
        - X: Attributes of the node.
        - y: dataset labels.
        - num_attr: the number of attributes that the algorithm chooses.
    Output
        - best_feature: The best feature in terms of information gain.
        - best_gain: The information gain value when the dataset is split by the best feature.
        
    Step 1: Choose 'num_attr' column names from X.columns without allowing replacement. Use np.random.choice. 
            Assign the result to 'attributes'
    Step 2: Set best_info_gain to zero and best_attr to None.
    Step 3: You should iterate the attributes chosen in Step 1.
            For each chosen attribute from Step 1, 'split' the dataset using the split function we have offered.
            Save the split attributes and labels.
    Step 4: Examine the information gain of the current trial. Use information_gain function you created.
    Step 5: Compare it to the current best gain, if the new gain is higher, reset best_gain and best_feature.
    Step 6: Return best_attr, best_info_gain.
    
    
    """
    # Step 1
    attributes = np.random.choice(X.columns[:-1], num_attr, replace=False) # CHANGE IT
    
    # Step 2
    best_info_gain = 0 # CHANGE IT
    best_attr = None
    
    # Step 3 - You should create a for loop and Step 4 and 5 will run inside the loop
    for attr in attributes:
        split_attr, split_labels = split(X,y,attr)
        # Step 4
        temp_gain = information_gain(y,split_labels)
        if temp_gain > best_info_gain:
            best_info_gain = temp_gain
            best_attr = attr
        # Step 5

    # Step 6
    return best_attr, best_info_gain

Try to find the best split of the playgolf dataset with `num_features` = 2. Report the best feature and best gain to `best_attr_playgolf` and `best_gain_playgolf` (0.4 pt).

In [40]:
np.random.seed(RANDOM_STATE)
best_attr_playgolf, best_gain_playgolf = best_split(playgolf,playgolf['Play Golf'],num_attr=2) # CHANGE IT

In [41]:
# TEST YOUR RESULT HERE
best_attr_playgolf, best_gain_playgolf

('Outlook', 0.11632653061224485)

Subtask 5: Now we have functions 1) to split the function based on one chosen feature (`split`) and 2) to choose the best feature to split (`find_best_split`). The next step will be to create one tree with all the information we have. Finish the function `build`. This function makes one tree of the random forest by using two previous functions. Since this function is a recursive one, it will return a complete tree, not a node.

In [ ]:
# 0.5 pt

def build(X, y, num_attr, tol=0.00001):
    """
    Input
        - X: Attributes of the data
        - y: dataset labels
        - num_attr: the number of attributes that the algorithm chooses for each node.
        - tol: information gain tolerance value.
    Output
        - node: a leaf or middle node.
        
    Step 1: Run the best split function to get the best attributes and the best information gain for the node.
    Step 2: Examine the best information gain value. If it is lower than the tolerance value (tol), 
            return the node with the best information gain value. The node should be a dictionary form 
            {"type": "leaf", "gain": the best information gain}.
    Step 3: If the best information gain is higher, split the dataset with the chosen best attribute.
    Step 4: Create an empty list called "branches" to save all the branches of the current node.
    Step 5: For each split attributes and labels, run this 'build' function recursively and store the result
            to the  "branches" list.
    Step 6: After all the recursion process is done, return the root node with its best attribute, branch information,
            and the best information gain.
    """
    
    # Step 1
    best_attr, best_info_gain = None, None # Change it

    # Step 2 - Change the if condition and the return value
    if None:
        return {} # CHANGE IT
    
    # Step 3
    split_attrs, split_labels = None, None # CHANGE IT
    
    # Step 4
    branches = []
    
    # Step 5 - You should create a for loop following the manual
    
    # Step 6 - Change None values to the correct values
    return {
        "type": "node",
        "best_feature": None,
        "branches": None,
        "value": None
    }

Bulid one tree forest using the whole playgolf dataset and report the tree to the variable 'single_tree'. Set the number of attributes to three. You do not need to specify the tolerance value (0.5 pt).

In [ ]:
np.random.seed(RANDOM_STATE)
single_tree = None # CHANGE IT

In [ ]:
single_tree

Subtask 6: Finally, it is time to generate a random forest with multiple trees. Complete `random_forest` function which chooses samples of the original dataset and create multiple trees forming a 'forest'.

In [ ]:
# 0.4 pt
import random 

def random_forest(X, y, num_tree, num_attr, tol = 0.00001):
    """
    Input
        - X: Attributes of the data
        - y: dataset labels
        - num_tree: the number of attributes that the algorithm chooses for each node.
        - num_attr: the number of attributes that the algorithm chooses for each node.
        - tol: information gain tolerance value.
    Output
        - trees: collection of trees
    
    Step 1: Create a list called 'trees' to save all trees generated during the process
    Step 2: Create a for loop iterating num_tree times.  Repeat Step 3 - 5 for 'num_tree' times in a for loop. 
            After the for loop finishes, trees list should have num_tree different trees inside.
    Step 3: For each loop, choose indices (not values) from the dataset X of the same size, but with allowing replacement.
            For example, you can pick [1,2,2,3,3] from the data [1,2,3,4,5]. 
            The size after random sampling should be the same as the vertical size of the dataset X.
    Step 4: Use the same indices as X to pick the labels from y with replacement. 
            If you use DataFrame, you may need to reset the indices (reset_index).
    Step 5: Build a tree using the subsets of X and y and save it to the list.
    Step 6: Return 'trees'.
    
    """
    
    trees = None
    return trees

Create one random forest on playgolf dataset. Set the number of tree to three, and the number of attributes to three as well. You do not need to change tolerance value from the default one (0.4 pt).

In [ ]:
np.random.seed(RANDOM_STATE)
forest = None # CHANGE IT

In [ ]:
forest

# 2. Evaluation 

#### Task 9: Accuracy, Precision, Recall, F1-score (0.3 pt)

We start to use our original dataset again! You will evaluate the random forest and the support vector machine classifier with various performance measures you have learned besides accuracy, such as precision, recall, and F1-score, also using scikit-learn. Here we continue to use the same dataset.

Your task is as follows:

1. Use standardized datasets (`X_train_st`, `y_train`...) throughout the task. To use the various score functions here, you need to convert the labels of `y_train` and `y_test` (`<=50K`, `>50K`) to numerical ones (0 or 1) since the score functions will not recognize the categorical labels. Create `y_train_numerical` and `y_test_numerical` with the converted labels (`<=50K` to 0, and `>50K` to 1). Refer to the previous labs. (0.1 pt)
2. Create an instance of an SVC classifier with the **polynomial** kernel.
3. Fit the model using the training set.
4. Report precision score, recall score, and F1-score using the test set, and save it into the variable `recall_score_svc`, `precision_score_svc`, and `f1_score_svc`. You can find out the information about the performance measures [here](https://scikit-learn.org/stable/modules/model_evaluation.html#precision-recall-f-measure-metrics). We require you to calculate the scores using the following functions in scikit-learn: `precision_score`, `recall_score`, and `f1_score`. There is no partial point if you are correct on only some of them (0.2 pt).

In [ ]:
# Subtask 1: 0.1 pt
y_train_numerical = pd.Series(0) # CHANGE IT
y_test_numerical = pd.Series(0) # CHANGE IT

Check if you successfully replaced the values here.

In [ ]:
y_train_numerical.unique(), y_test_numerical.unique()

In [ ]:
# Subtask 2: 0.2 pt
recall_score_svc = None # CHANGE IT
precision_score_svc = None # CHANGE IT
f1_score_svc = None # CHANGE IT

Print three scores here.

In [ ]:
precision_score_svc, recall_score_svc, f1_score_svc

#### Task 10: AUC / AUPRC (0.3 pt)

You will evaluate the random forest and the support vector machine classifier with various performance measures related to the ROC curve, such as the area under the ROC curve (AUC) and rea under the precision-recall curve (AUPRC).

Your task is as follows:

1. Use the same dataset as the ones for Task 7 (`X_train_st` and `y_train_numerical`). AUPRC and AOC score also only recognize numerical labels.
2. Create an instance of a random forest classifier without setting any constraint. Don't forget to set the random state to our value `RANDOM_STATE`.
3. Fit the model on the training set.
4. Print the accuracy on the test set to `accuracy_rf` (0.1 pt).
4. Report AUC and AUPRC using the test set, and save it into the variable called *auc_rf, auprc_rf*. You can find out the information about the performance measures [here](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.roc_auc_score.html#sklearn.metrics.roc_auc_score) for AUC score, and [here](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.average_precision_score.html) for AUPRC score. AUPRC has many names, and it is supported as *average precision score* in scikit-learn. We require you to calculate the scores using the following functions: *roc_auc_score, average_precision_score*. There is no partial point if you are correct on only some of them (0.2 pt).

In [ ]:
accuracy_rf = None # CHANGE IT
auc_rf = None # CHANGE IT
auprc_rf = None # CHANGE IT

Print your scores here

In [ ]:
(accuracy_rf, auc_rf, auprc_rf)

#### Task 11: Apply them together with scikit-learn (0.4 pt)

Here you will try to apply the grid search using the performance measures you have tried on Task 5 and Task 6, and pick the best performing model in terms of specific performance measures.

Our dataset is imbalanced, meaning that the healthy patient is dominant. Therefore, we can expect that the best model can be different, and we may also need to use AUPRC to get the most suitable model. 

Your task is as follows:

1. Use the same dataset as the ones for Task 7 (`X_train_st` and `y_train_numerical`). AUPRC and AOC score also only recognize numerical labels.
2. Create an instance of a kNN classifier without setting any constraint. 
3. Run grid search with a dictionary stating n_neighbors from 1 to 10, and use two different scoring measures: AUPRC (average_precision) and F1-score (f1). 
4. Put the best classifiers into the respective variable called *auprc_best_classifier* and *f1_best_classifier*. Set cv=5 for grid search cross-validation. 

If you complete the method, you can run the following line to check whether your functions are correct or not. Note that we will evaluate your functions with different data, so be careful to implement them.

In [ ]:
auprc_best_classifier = None # CHANGE IT
f1_best_classifier = None # CHANGE IT

Check your scores here!

In [ ]:
(auprc_best_classifier, f1_best_classifier)

#### Task 12: Task 5 implementation (0.8 pt)

This task requires you to implement the following performance measures:
 - Accuracy (0.2 pt)
 - Precision (0.2 pt)
 - Recall (0.2 pt)
 - F1-score (0.2 pt)
 
All inputs will be the NumPy arrays, so you can use any NumPy array methods to calculate the scores.

In [ ]:
def accuracy_manual(truth, predicted):
    return None # CHANGE IT

In [ ]:
def precision_manual(truth, predicted):
    return None # CHANGE IT

In [ ]:
def recall_manual(truth, predicted):
    return None # CHANGE IT

In [ ]:
def f1_score_manual(truth, predicted):
    return None # CHANGE IT

Assign the results of your four function on two arrays (`truth`, `predicted`).

In [ ]:
truth = [1,0,0,0,1,1,1,0,0,1,1,0]
predicted = [1,0,1,0,1,1,0,1,1,0,1,1]

In [ ]:
accuracy_score_manual = None # CHANGE IT
precision_score_manual = None # CHANGE IT
recall_score_manual = None # CHANGE IT
f1_score_manual = None # CHANGE IT

Show your results here!

In [ ]:
(accuracy_score_manual, precision_score_manual, recall_score_manual, f1_score_manual)

# 3. Deployment

**Task 13: Save models into a file using pickle (0.3 pt)**

In this task, you need to pick the best model using cross-validation and deploy it as a pickle file. For this task, we will use the diabetes data that we used for Homework 1. This dataset is originally from the National Institute of Diabetes and Digestive and Kidney Diseases. The objective of the dataset is to diagnostically predict whether or not a patient has diabetes, based on certain diagnostic measurements included in the dataset. Several constraints were placed on the selection of these instances from a larger database. In particular, all patients here are females at least 21 years old of Pima Indian heritage. You can find it in the homework folder.

- Load the dataset into `diabetes`.

In [ ]:
diabetes = pd.read_csv("datasets/diabetes.csv")

In [ ]:
diabetes.head()

- Split the dataset into two parts: attributes (`X`) and labels (Outcome, `y`).

In [ ]:
X = None # CHANGE IT
y = None # CHANGE IT

Your task is as follows:

1. Create an instance of an SVC classifier without setting any constraint.
2. Run grid search with a list of two dictionaries. In the first dictionary, you should examine 'poly' kernel, with degree = [2, 3, 4]. In the second dictionary, you should test two kernels ['linear', 'rbf'] with a list of C values [10, 100]. Use **AUPRC** as its scoring measure. Set cv=5 for grid search cross-validation. Since grid search uses stratified k-fold inside, you should put the complete dataset.
3. Save the best classifier into the variable called `svm_best_classifier_2` and save the trained model into `model_diabetes.pickle` using pickle. When saving your model, do not specify any folder.
 - **Do not use your own specific name for the model!**

In [ ]:
svm_best_classifier_2 = None # CHANGE IT

Show your classifier here!

In [ ]:
svm_best_classifier_2

**Task 14: DASH deployment (0.3 pt)**

You will run DASH application by using the model `model_diabetes.pickle` you exported with the project files in the `webplatform_dash` folder. Locate the model in the same folder with this jupeter notebook, and go into `webplatform_dash` folder. There you can run your own DASH application as you learned from Lab 5. Note that you should **not** move the model file into `webplatform_dash`.

- Submit a screenshot with your model file into one zip file in a separate submission form for your DASH project. For the details of the DASH deployment, check out Lab 5.